# Analysing mock datasets

In [2]:
# packages

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

import sys
sys.path.append("../src")

from analysis import *
from inference import *

In [8]:
import re

def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower() 
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)] 
    return sorted(l, key=alphanum_key)

In [13]:
datapath = "../experiment_outputs/test/datasets/"
datafiles = os.listdir(datapath)
metadatafiles = [f"metadata{i.split('dataset')[1].split('csv')[0]}txt"\
                for i in datafiles]

datafiles = natural_sort(datafiles)
metadatafiles = natural_sort(metadatafiles)

In [14]:
df = pd.read_csv(datapath+datafiles[0], index_col=0)
metatext = open(f"{datapath}../metadata/"+metadatafiles[0], "r").read().split("\n")
df

,dataset,measurement_noise,time,dt,sp1,sp2
0,0.0,0.0,0.000000,7.50000,0.432183,0.441384
1,0.0,0.0,7.500000,7.50000,0.448945,0.495039
2,0.0,0.0,15.000000,7.50000,0.449328,0.495318
3,0.0,0.0,22.500000,7.50000,0.449331,0.495320
4,0.0,0.0,30.000000,NaN,0.449331,0.495320
...,...,...,...,...,...,...
14795,399.0,0.2,28.787879,0.30303,0.492285,0.444287
14796,399.0,0.2,29.090909,0.30303,0.410269,0.403906
14797,399.0,0.2,29.393939,0.30303,0.528283,0.547213
14798,399.0,0.2,29.696970,0.30303,0.399044,0.439545


In [15]:
def get_meta(metatext):
    meta = {}
    for n in range(len(metatext)):
        line = metatext[n]
        if "parameters" in line:
            meta["parameters"] = np.array([np.float64(j) for j in line.split("parameters: ")[1].split(",")])
            break

    meta["init_cond"] = np.array([[np.float64(i) for i in metatext[j].split(",")] for j in range(1, n)])

    for i in range(n+1, len(metatext)-1):
        key, val = metatext[i].split(": ")
        meta[key] = np.float64(val)

    key, val = metatext[-1].split(": ")
    meta[key] = bool(val)

    return meta

In [16]:
df

,dataset,measurement_noise,time,dt,sp1,sp2
0,0.0,0.0,0.000000,7.50000,0.432183,0.441384
1,0.0,0.0,7.500000,7.50000,0.448945,0.495039
2,0.0,0.0,15.000000,7.50000,0.449328,0.495318
3,0.0,0.0,22.500000,7.50000,0.449331,0.495320
4,0.0,0.0,30.000000,NaN,0.449331,0.495320
...,...,...,...,...,...,...
14795,399.0,0.2,28.787879,0.30303,0.492285,0.444287
14796,399.0,0.2,29.090909,0.30303,0.410269,0.403906
14797,399.0,0.2,29.393939,0.30303,0.528283,0.547213
14798,399.0,0.2,29.696970,0.30303,0.399044,0.439545


In [17]:
get_meta(metatext)

{'parameters': array([ 0.5488135 ,  0.71518937, -1.6740246 ,  0.4105985 ,  0.14404357,
        -1.57456311]),
 'init_cond': array([[0.43218286, 0.44138427],
        [0.89498139, 0.30381701],
        [0.39472457, 0.19375925],
        [0.05414083, 0.55097219],
        [0.57845904, 0.20939048],
        [1.53220055, 0.12780951],
        [0.32796392, 0.30762591],
        [0.91931407, 0.96982179],
        [0.35374899, 0.45520721],
        [0.17171224, 0.08873434],
        [0.249643  , 0.3903342 ],
        [0.74542815, 0.80597816],
        [0.24291504, 0.28403292],
        [0.15360505, 0.13088892],
        [0.09736724, 1.35398166],
        [0.22316742, 0.25852192],
        [0.13332832, 0.60036973],
        [0.10380535, 0.3022245 ],
        [0.17080048, 0.45797327],
        [0.22298914, 0.15451305]]),
 'env_noise': 0.0,
 't0': 0.0,
 'dt': 0.001,
 'seed': 0.0,
 'repetitions': 1.0,
 'scale_meas_noise_by_abund': True}

In [44]:
datasets = [df[df.dataset==i] for i in df.dataset.unique()]